In [1]:
import glob

import pandas as pd
import dask.dataframe as dd

In [2]:
list_parquet = glob.glob("..\\full_data\\figma_plan\\*.parquet")

df = pd.DataFrame()
for path in list_parquet:
    part_df = dd.read_parquet(path).compute()
    df = pd.concat([df,part_df],axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415317 entries, 0 to 415316
Columns: 3036 entries, user_id to other_male_urls_frac
dtypes: category(26), float32(3007), int32(1), int8(2)
memory usage: 4.7 GB
